# Sales Data Analysis Project with Tableau

# $\to$

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [2]:

# 1. Capture the dictionary of DataFrames
data = pd.read_excel("./AmazingMartEU2.xlsx", sheet_name=['ListOfOrders','OrderBreakdown','SalesTargets'])

# 2. Assign the DataFrames to specific variables
df_orders = data['ListOfOrders']
df_breakdown = data['OrderBreakdown']
df_targets = data['SalesTargets']
dfs=[df_orders,df_breakdown,df_targets]


In [3]:
print(dfs[0].isna().sum())
print("Number of duplication: ",df_orders.duplicated().sum())


Order ID         0
Order Date       0
Customer Name    0
City             0
Country          0
Region           0
Segment          0
Ship Date        0
Ship Mode        0
State            0
dtype: int64
Number of duplication:  0


In [4]:
print(df_breakdown.isna().sum())
print("Number of duplication: ",df_breakdown.duplicated().sum())


Order ID        0
Product Name    0
Discount        0
Sales           0
Profit          0
Quantity        0
Category        0
Sub-Category    0
dtype: int64
Number of duplication:  2


In [5]:
df_breakdown.drop_duplicates(inplace=True)

In [6]:
# OrderBreakdown
df = df_orders.merge(df_breakdown, on='Order ID', how='inner')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8045 entries, 0 to 8044
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       8045 non-null   object        
 1   Order Date     8045 non-null   datetime64[ns]
 2   Customer Name  8045 non-null   object        
 3   City           8045 non-null   object        
 4   Country        8045 non-null   object        
 5   Region         8045 non-null   object        
 6   Segment        8045 non-null   object        
 7   Ship Date      8045 non-null   datetime64[ns]
 8   Ship Mode      8045 non-null   object        
 9   State          8045 non-null   object        
 10  Product Name   8045 non-null   object        
 11  Discount       8045 non-null   float64       
 12  Sales          8045 non-null   int64         
 13  Profit         8045 non-null   int64         
 14  Quantity       8045 non-null   int64         
 15  Category       8045 n

In [8]:
df.columns.tolist()

['Order ID',
 'Order Date',
 'Customer Name',
 'City',
 'Country',
 'Region',
 'Segment',
 'Ship Date',
 'Ship Mode',
 'State',
 'Product Name',
 'Discount',
 'Sales',
 'Profit',
 'Quantity',
 'Category',
 'Sub-Category']

In [9]:
df.duplicated().sum()

np.int64(0)

In [10]:
df.describe().T.round(2)[2:]

,count,mean,min,25%,50%,75%,max,std
Discount,8045.0,0.110075,0.0,0.0,0.0,0.1,0.85,0.181787
Sales,8045.0,291.90317,3.0,48.0,117.0,313.0,6517.0,485.258616
Profit,8045.0,35.202237,-3060.0,1.0,14.0,47.0,2476.0,178.147747
Quantity,8045.0,3.772281,1.0,2.0,3.0,5.0,14.0,2.20361


In [11]:
df.head(2)

,Order ID,Order Date,Customer Name,City,Country,Region,Segment,Ship Date,Ship Mode,State,Product Name,Discount,Sales,Profit,Quantity,Category,Sub-Category
0,BN-2011-7407039,2011-01-01,Ruby Patel,Stockholm,Sweden,North,Home Office,2011-01-05,Economy Plus,Stockholm,"Enermax Note Cards, Premium",0.5,45,-26,3,Office Supplies,Paper
1,AZ-2011-9050313,2011-01-03,Summer Hayward,Southport,United Kingdom,North,Consumer,2011-01-07,Economy,England,"Dania Corner Shelving, Traditional",0.0,854,290,7,Furniture,Bookcases


In [12]:
# check if all product name contain comma 
df['Product Name'].str.contains(',').all()

np.True_

In [13]:
# [{feature Engineering}]
df['product_description'] = df['Product Name'].str.split(',', expand=True)[1].str.strip()
df['Product_main']=df['Product Name'].str.split(',', expand=True)[0].str.strip()

### Shipping period days from dates

In [14]:
# [{feature Engineering}]
df['Shipping_period_days'] = (df['Ship Date'] - df['Order Date']).dt.days

 ### get the ***price*** of the product **before/after** discount

In [15]:
# [{feature Engineering}]
df['price_after_dis']= (df['Sales'] / df['Quantity']).round(2)
df['original_price'] = (df['price_after_dis'] / (1 - df['Discount'])).round(2)

## Uni-Variate Analysis

### `Order ID`

- Customer analysis: How many orders per customer?
- Profitability per order:
- Number of products per order: How many orders have 1, 2, 3… products?

In [16]:
print("number of orders =", df['Order ID'].nunique())

number of orders = 4117


In [17]:
print("Number of products per order =",set(df['Order ID'].value_counts().values.tolist()))

Number of products per order = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


### `Order Date`

In [18]:
df['Order Date'].dt.year.value_counts().sort_index()

Order Date
2011    1462
2012    1870
2013    2100
2014    2613
Name: count, dtype: int64

In [19]:
# [{feature Engineering}]
df['Order Month'] = df['Order Date'].dt.month_name()
df['Order Year'] = df['Order Date'].dt.year
df['Order Day'] = df['Order Date'].dt.day_name()

In [20]:
# [{feature Engineering}]
# order months name in correct order
month_order = ['January', 'February', 'March', 'April', 'May', 'June','July', 'August', 'September', 'October', 'November', 'December']
df['Order Month'] = pd.Categorical(df['Order Month'], categories=month_order, ordered=True)

In [21]:

months = df['Order Month'].value_counts().sort_index()
plt.figure(figsize=(12,6))
fig = px.line(
    x=months.index,
    y=months.values,
    markers=True,
    labels={'x': 'Month', 'y': 'Number of Orders'},
    title='Number of Orders per month'
)



fig.show()


<Figure size 1200x600 with 0 Axes>

In [22]:

months = df['Order Month'].value_counts().reindex(month_order)
plt.figure(figsize=(12,6))
fig = px.bar(
    x=months.index,
    y=months.values,
    labels={'x': 'Month', 'y': 'Number of Orders'},
    title='Number of Orders per Month'
)

fig.show()

<Figure size 1200x600 with 0 Axes>

In [23]:
# years=df['Order Year'].value_counts().sort_index()
# sns.lineplot(x=years.index, 
#              y=years.values,
#              markers='o')
# plt.title('Number of Orders per Year')
# plt.xlabel('Year')
# plt.ylabel('Number of Orders')
# plt.xticks(years.index.astype(int)) 
# plt.show()

In [24]:
years = df['Order Year'].value_counts().sort_index()
plt.figure(figsize=(12,6))
fig = px.line(
    x=years.index.astype(str),
    y=years.values,
    markers=True,
    labels={'x': 'Year', 'y': 'Number of Orders'},
    title='Number of Orders per Year'
)



fig.show()


<Figure size 1200x600 with 0 Axes>

In [25]:

monthly_orders = df['Order Date'].dt.to_period('M').astype(str).value_counts().sort_index()
plt.figure(figsize=(12,6))
fig = px.line(
    x=monthly_orders.index,
    y=monthly_orders.values,
    title='Monthly Orders Over Time',
    labels={'x': 'Year-Month', 'y': 'Number of Orders'},
    markers=True,
    
)

fig.show()


<Figure size 1200x600 with 0 Axes>

### `Customer Name`

Number of Customers

In [26]:
print("Number of Customer =",df['Customer Name'].nunique())

Number of Customer = 792


### `City`

In [27]:
df['City'].nunique()

999

In [28]:
df[df['City']=='London']['Order ID'].nunique()

113

Number of Customers per Country: How many customers from each country?

In [29]:
df.groupby('Customer Name')['Country'].first().value_counts()

Country
Germany           167
France            166
United Kingdom    138
Italy             101
Spain              67
Netherlands        42
Austria            34
Sweden             16
Switzerland        13
Finland             9
Ireland             8
Belgium             8
Norway              8
Portugal            8
Denmark             7
Name: count, dtype: int64

Highest Selling Products: What are the top 10 products by quantity sold?

In [30]:
df.groupby(df[df['Country'].isin(['Italy'])]['Product_main'])['Quantity'].sum().sort_values(ascending=False).head(10)

Product_main
Rogers Shelving             46
Fellowes Trays              40
Rogers File Cart            32
Stanley Canvas              30
Stanley Pencil Sharpener    29
Eldon Lockers               29
Acco 3-Hole Punch           29
Fellowes Box                28
BIC Markers                 28
Apple Signal Booster        28
Name: Quantity, dtype: int64

In [36]:
# Group profit by month (same as orders, but sum instead of count)
monthly_profit = df.groupby(df['Order Date'].dt.to_period('M'))['Profit'].sum()

# Convert PeriodIndex to string for Plotly
monthly_profit.index = monthly_profit.index.astype(str)
plt.figure(figsize=(12,6))
# Plot
fig = px.line(
    x=monthly_profit.index,
    y=monthly_profit.values,
    title='Monthly Profit Over Time',
    labels={'x': 'Year-Month', 'y': 'Total Profit'},
    markers=True
)

fig.show()


<Figure size 1200x600 with 0 Axes>

In [38]:
df[df['Order Date'].dt.to_period('M')=='Aug 2012']['Discount'].value_counts().sort_index()

Discount
0.00    176
0.10     48
0.15     12
0.20      2
0.35      1
0.40      5
0.45      1
0.50     31
0.60      1
Name: count, dtype: int64

In [43]:
df[df['Order Date'].dt.to_period('M')=='Jan 2011']['Discount'].value_counts().sort_index()

Discount
0.0    42
0.1     5
0.4     2
0.5    13
0.6     4
Name: count, dtype: int64

In [50]:
df[df['Discount']!=0].groupby('Discount')['Profit'].sum().sort_index()

Discount
0.10    95569
0.15    16649
0.20     1705
0.30     -635
0.35    -8330
0.40   -19268
0.45    -1104
0.50   -86458
0.60   -14949
0.65    -5260
0.70    -5497
0.80     -426
0.85    -1925
Name: Profit, dtype: int64

In [57]:
discount_profit = df[df['Discount']!=0].groupby('Discount')['Profit'].sum().sort_index()

px.line(
    x=discount_profit.index,
    y=discount_profit.values,
    title="Total Profit for Each Discount Level",
    labels={'x': 'Discount', 'y': 'Total Profit'},
    markers=True
).show()


> ### 📌 The worst discount of all:
- 0.50 → a massive loss of -86,458, even though the number of orders is high (982 orders).
- This means that the “50% off” discount is not profitable at all.

In [56]:
df[df['Discount']!=0]['Discount'].value_counts().sort_index()

Discount
0.10    1315
0.15     287
0.20     110
0.30      40
0.35      37
0.40     245
0.45       2
0.50     982
0.60      96
0.65      14
0.70       6
0.80       3
0.85       1
Name: count, dtype: int64

In [58]:
discount_count = df[df['Discount']!=0]['Discount'].value_counts().sort_index()

px.bar(
    x=discount_count.index,
    y=discount_count.values,
    title="Number of Times Each Discount Was Applied",
    labels={'x': 'Discount', 'y': 'Count'}
).show()


### `Discount`
Avg Profit per order per Discount Rate

In [59]:
discount_profit = df[df['Discount'] != 0].groupby('Discount')['Profit'].mean().sort_index()

discount_profit


Discount
0.10      72.676046
0.15      58.010453
0.20      15.500000
0.30     -15.875000
0.35    -225.135135
0.40     -78.644898
0.45    -552.000000
0.50     -88.042770
0.60    -155.718750
0.65    -375.714286
0.70    -916.166667
0.80    -142.000000
0.85   -1925.000000
Name: Profit, dtype: float64

In [60]:
import plotly.express as px

disc_df = df[df['Discount'] != 0].groupby('Discount')['Profit'].mean().reset_index()

fig = px.line(
    disc_df,
    x='Discount',
    y='Profit',
    markers=True,
    title='Average Profit per Discount Level',
    labels={'Discount': 'Discount Level', 'Profit': 'Average Profit'}
)

fig.show()


In [62]:
pivot = df[df['Discount'] != 0].pivot_table(
    index='Discount',
    values='Profit',
    aggfunc=['sum', 'count']
)

pivot.columns = ['Total Profit', 'Order Count']  # rename columns
pivot = pivot.sort_index()
pivot


,Total Profit,Order Count
Discount,,
0.10,95569,1315
0.15,16649,287
0.20,1705,110
0.30,-635,40
0.35,-8330,37
0.40,-19268,245
0.45,-1104,2
0.50,-86458,982
0.60,-14949,96


Discount
0.10      72.676046
0.15      58.010453
0.20      15.500000
0.30     -15.875000
0.35    -225.135135
0.40     -78.644898
0.45    -552.000000
0.50     -88.042770
0.60    -155.718750
0.65    -375.714286
0.70    -916.166667
0.80    -142.000000
0.85   -1925.000000
Name: Profit, dtype: float64

In [64]:
loss_ratio = (
    df[df['Discount'] != 0]
    .assign(is_loss=lambda x: x['Profit'] < 0)
    .groupby('Discount')['is_loss']
    .mean()
    .sort_index()
)

loss_ratio


Discount
0.10    0.202281
0.15    0.376307
0.20    0.400000
0.30    0.650000
0.35    0.783784
0.40    0.779592
0.45    1.000000
0.50    0.965377
0.60    1.000000
0.65    1.000000
0.70    1.000000
0.80    1.000000
0.85    1.000000
Name: is_loss, dtype: float64

KeyError: 'is_loss'